## Web References

### System

- [How to copy a file without using scp inside an ssh session?](https://superuser.com/questions/291423/how-to-copy-a-file-without-using-scp-inside-an-ssh-session)

### PySpark

- [Complete Machine Learning Project with PySpark MLlib Tutorial](https://www.youtube.com/watch?v=1a7bB1ZcZ3k)
- [The ONLY PySpark Tutorial You Will Ever Need.](https://www.youtube.com/watch?v=cZS5xYYIPzk)
- [PySpark When Otherwise | SQL Case When Usage](https://sparkbyexamples.com/pyspark/pyspark-when-otherwise/)
- [Spark rlike() Working with Regex Matching Examples](https://sparkbyexamples.com/spark/spark-rlike-regex-matching-examples/)
- [spark dataframe drop duplicates and keep first](https://stackoverflow.com/questions/38687212/spark-dataframe-drop-duplicates-and-keep-first)

### Anomaly Detection

- [How to Build an Anomaly Detection Engine with Spark, Akka and Cassandra](https://learning.oreilly.com/videos/how-to-build/9781491955253/9781491955253-video244545/)
- [Real Time Detection of Anomalies in the Database Infrastructure using Apache Spark](https://www.youtube.com/watch?v=1IsMMmug5q0)

### Other

- [What is CRISP DM?](https://www.datascience-pm.com/crisp-dm-2/)

### Internet Traffic

- [Data mining approach for predicting the daily Internet data traffic of a smart university](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0176-5)

## Import Libraries

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, asc, desc, col

## HDFS Preparation

In [2]:
%%bash
#!/bin/bash

# delete the 
if hadoop fs -test -d router; then
    # delete the output directory
    hadoop fs -rm -r router/output

    # create a new output directory
    hadoop fs -mkdir router/output
else
    # create the router directory and upload the input files
    hadoop fs -mkdir router
    hadoop fs -mkdir router/sample
    hadoop fs -put data/bandwidth.csv router/sample/

    hadoop fs -put data/raw router/raw

    # create the output directory
    hadoop fs -mkdir router/output
fi

hadoop fs -ls router/raw

Found 72 items
-rw-r--r--   3 jfoul001 users     150806 2022-03-05 10:07 router/raw/20220304T2210.csv.gz
-rw-r--r--   3 jfoul001 users     149945 2022-03-05 10:07 router/raw/20220304T2220.csv.gz
-rw-r--r--   3 jfoul001 users     149142 2022-03-05 10:07 router/raw/20220304T2230.csv.gz
-rw-r--r--   3 jfoul001 users     151781 2022-03-05 10:07 router/raw/20220304T2240.csv.gz
-rw-r--r--   3 jfoul001 users     150343 2022-03-05 10:07 router/raw/20220304T2250.csv.gz
-rw-r--r--   3 jfoul001 users     149798 2022-03-05 10:07 router/raw/20220304T2300.csv.gz
-rw-r--r--   3 jfoul001 users     148781 2022-03-05 10:07 router/raw/20220304T2310.csv.gz
-rw-r--r--   3 jfoul001 users     148639 2022-03-05 10:07 router/raw/20220304T2320.csv.gz
-rw-r--r--   3 jfoul001 users     149012 2022-03-05 10:07 router/raw/20220304T2330.csv.gz
-rw-r--r--   3 jfoul001 users     150019 2022-03-05 10:07 router/raw/20220304T2340.csv.gz
-rw-r--r--   3 jfoul001 users     150122 2022-03-05 10:07 router/raw/20220304T2350.cs

## Initialize the Spark Session

In [3]:
spark = SparkSession.builder.appName('cw02').getOrCreate()
spark

Setting default log level to "ERROR".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## I. Data Understanding

Identify, collect, and analyze the data sets that will help accomplish the project goals

### A. Collect Initial Data

In [4]:
# read the full batch to process
df_batch = spark.read.csv('router/raw/', header=False, inferSchema=True).toDF('Direction','Interval Length','Intervals Saved','IP','Interval Start','Interval End','Bytes Used')

In [5]:
# drop the intervals saved column
df_batch = df_batch.drop('Intervals Saved')

### B. Describe data

Examine the data and document its surface properties like data format, number of records, or field identities.

#### 1. Data Format

In [6]:
df_batch.printSchema()

root
 |-- Direction: string (nullable = true)
 |-- Interval Length: string (nullable = true)
 |-- IP: string (nullable = true)
 |-- Interval Start: string (nullable = true)
 |-- Interval End: integer (nullable = true)
 |-- Bytes Used: long (nullable = true)



In [7]:
df_batch.count()

1487304

#### 2. Remove Duplicates

Many duplicate records are present in the raw data since the data snapshots overlaps.

In [8]:
# drop duplicates
df_unique = df_batch.drop_duplicates()
df_unique.count()

397016

There are duplicates where only the `Bytes Used` field is different due to when the data snapshot was taken.

In [9]:
#df_max_bytes = df_unique.groupBy('IP').agg({"Bytes Used": "max"})
df_unique = df_unique.groupBy(['Direction','Interval Length','IP','Interval Start','Interval End']) \
    .max('Bytes Used') \
    .withColumnRenamed('max(Bytes Used)', 'Bytes Used')

df_unique.count()

385863

In [10]:
df_unique.show(5)

+---------+---------------+-------------+--------------+------------+----------+
|Direction|Interval Length|           IP|Interval Start|Interval End|Bytes Used|
+---------+---------------+-------------+--------------+------------+----------+
| download|         minute|192.168.3.152|    1646456160|  1646456220|         0|
|     down|       dclass_5|          449|      COMBINED|  1646439806|1646439808|
|     down|       dclass_5|          449|      COMBINED|  1646444020|1646444022|
|     down|       dclass_1|          449|      COMBINED|  1646436902|1646436904|
|     down|       dclass_8|          449|      COMBINED|  1646434402|1646434404|
+---------+---------------+-------------+--------------+------------+----------+
only showing top 5 rows



#### 2. Unique Categories

In [11]:
# if an IP address is stored in the IP column recode it as simply 'IP'
df_interval_type = df_unique.withColumn('interval_type', 
    when(df_unique['IP']
    .rlike('^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$'), 'IP')
    .otherwise(df_unique['IP'])
)

df_interval_type.groupBy(['interval_type']).count().show()

+-------------+------+
|interval_type| count|
+-------------+------+
|           15|  5257|
|           31|   325|
|          449|148959|
|           24|   824|
|           IP|181600|
|           12|    71|
|     COMBINED| 48827|
+-------------+------+



In [12]:
# record count per catergory
df_interval_type.groupBy(['Interval Length', 'interval_type']) \
    .count() \
    .orderBy(['Interval Length', 'interval_type',], ascending=True) \
    .show(df_interval_type.count())

+---------------+-------------+------+
|Interval Length|interval_type| count|
+---------------+-------------+------+
|            180|     COMBINED|  1066|
|              2|     COMBINED| 43665|
|              2|           IP|135533|
|           7200|     COMBINED|   744|
|            900|     COMBINED|   242|
|            900|           IP|  5803|
|            day|     COMBINED|   732|
|            day|           IP|  3412|
|       dclass_1|           12|    13|
|       dclass_1|           15|   726|
|       dclass_1|           24|    79|
|       dclass_1|           31|    32|
|       dclass_1|          449| 21751|
|       dclass_2|           12|    13|
|       dclass_2|           15|   726|
|       dclass_2|           24|    79|
|       dclass_2|           31|    32|
|       dclass_2|          449| 21751|
|       dclass_3|           12|     3|
|       dclass_3|           15|    72|
|       dclass_3|           24|    65|
|       dclass_3|           31|    32|
|       dclass_3|        

#### 3. How many intervals are available for the various intervals?

This number should be less as for every interval there could be multiple IP addresses (where `interval_type == 'IP'`).

In [13]:
df_interval_type.drop_duplicates(['Direction', 'Interval Length', 'interval_type', 'Interval Start', 'Interval End']) \
    .groupBy(['Interval Length', 'interval_type']) \
    .count() \
    .orderBy(['Interval Length', 'interval_type',], ascending=True) \
    .show(df_interval_type.count())

+---------------+-------------+-----+
|Interval Length|interval_type|count|
+---------------+-------------+-----+
|            180|     COMBINED| 1066|
|              2|     COMBINED|43665|
|              2|           IP|43688|
|           7200|     COMBINED|  744|
|            900|     COMBINED|  242|
|            900|           IP|  242|
|            day|     COMBINED|  732|
|            day|           IP|   64|
|       dclass_1|           12|   13|
|       dclass_1|           15|  726|
|       dclass_1|           24|   79|
|       dclass_1|           31|   32|
|       dclass_1|          449|21751|
|       dclass_2|           12|   13|
|       dclass_2|           15|  726|
|       dclass_2|           24|   79|
|       dclass_2|           31|   32|
|       dclass_2|          449|21751|
|       dclass_3|           12|    3|
|       dclass_3|           15|   72|
|       dclass_3|           24|   65|
|       dclass_3|           31|   32|
|       dclass_3|          449|   72|
|       dcla